# Comparison between ctypes and Cython

@author: Adrian Oeftiger @date: 11.07.2018

We compare calling a shared `C` library in Python interfaced via the `ctypes` library and via `Cython`.

In [1]:
from __future__ import division, print_function

import numpy as np
from scipy.constants import c, e
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(0)

In [2]:
# INPUTS
n_mp = int(1e6)

dE = np.random.randn(n_mp) * 1e6 # [eV]
dt = np.random.randn(n_mp) * 1e-7 # [s]

gamma = 1.5

beta_x = 15
beta_y = 20
D_x = 2
epsn_x = 2e-6
epsn_y = 2e-6

In [3]:
# COMPUTED QUANTITIES
beta = np.sqrt(1 - gamma**-2)
dp = dE / (beta * c)
mu_x = np.random.randn(n_mp) * np.sqrt(epsn_x)
mu_y = np.random.randn(n_mp) * np.sqrt(epsn_y)

x = np.sqrt(beta_x) * mu_x + D_x * dp
y = np.sqrt(beta_y) * mu_y
z_beamframe = (-dt * beta * c) * gamma # Lorentz transformed

In [4]:
n_nodes = 64, 64, 64
dx, dy, dz = map(
    lambda ary, n: (np.max(ary) - np.min(ary)) / (n - 3), 
    [x, y, z_beamframe], n_nodes)
x0, y0, z0 = map(
    lambda ary, d: ary.min() - d,
    [x, y, z_beamframe], [dx, dy, dz])

In [5]:
%load_ext Cython

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
# compilable with $gcc -std=c99 -O3 -fPIC -shared -o get_weights_c.so get_weights_c.c -lm 
with open('get_weights_c.c', 'wt') as fh:
    fh.write('''
#include <math.h>

void get_weights_c(
    // inputs:
    double* x, double* y, double* z,
    int n,
    // outputs:
    double* weight_ijk, double* weight_i1jk,
    double* weight_ij1k, double* weight_i1j1k,
    double* weight_ijk1, double* weight_i1jk1,
    double* weight_ij1k1, double* weight_i1j1k1
) {{
    double jj, ii, kk;
    double dxi, dyi, dzi;
    for (int i = 0; i < n; i++) {{
        // indices
        double jj = floor((x[i] - {x0:{prec}}) / {dx:{prec}});
        double ii = floor((y[i] - {y0:{prec}}) / {dy:{prec}});
        double kk = floor((z[i] - {z0:{prec}}) / {dz:{prec}});

        // distances
        double dxi = x[i] - ({x0:{prec}} + jj * {dx:{prec}});
        double dyi = y[i] - ({y0:{prec}} + ii * {dy:{prec}});
        double dzi = z[i] - ({z0:{prec}} + kk * {dz:{prec}});

        // weights
        weight_ijk[i] =    (1.-dxi/{dx:{prec}})*(1.-dyi/{dy:{prec}})*(1.-dzi/{dz:{prec}});
        weight_i1jk[i] =   (1.-dxi/{dx:{prec}})*(dyi/{dy:{prec}})   *(1.-dzi/{dz:{prec}});
        weight_ij1k[i] =   (dxi/{dx:{prec}})   *(1.-dyi/{dy:{prec}})*(1.-dzi/{dz:{prec}});
        weight_i1j1k[i] =  (dxi/{dx:{prec}})   *(dyi/{dy:{prec}})   *(1.-dzi/{dz:{prec}});
        weight_ijk1[i] =   (1.-dxi/{dx:{prec}})*(1.-dyi/{dy:{prec}})*(dzi/{dz:{prec}});
        weight_i1jk1[i] =  (1.-dxi/{dx:{prec}})*(dyi/{dy:{prec}})   *(dzi/{dz:{prec}});
        weight_ij1k1[i] =  (dxi/{dx:{prec}})   *(1.-dyi/{dy:{prec}})*(dzi/{dz:{prec}});
        weight_i1j1k1[i] = (dxi/{dx:{prec}})   *(dyi/{dy:{prec}})   *(dzi/{dz:{prec}});
    }}
}}
    '''.format(
        x0=x0, y0=y0, z0=z0, 
        dx=dx, dy=dy, dz=dz,
        prec=".17")
    )

# Memory allocation by itself

In [8]:
%%timeit -n 10

global weight_ijk, weight_i1jk, weight_ij1k, weight_i1j1k, \
weight_ijk1, weight_i1jk1, weight_ij1k1, weight_i1j1k1

(weight_ijk, weight_i1jk, weight_ij1k, weight_i1j1k,
weight_ijk1, weight_i1jk1, weight_ij1k1, weight_i1j1k1) = (
    np.empty_like(x) for _ in range(8)
)

10 loops, best of 3: 27.2 µs per loop


$\implies$ negligible impact on timing

# ctypes

In [9]:
!gcc -std=c99 -O3 -fPIC -shared -o get_weights_c.so get_weights_c.c -lm

In [10]:
import ctypes

In [11]:
from numpy.ctypeslib import ndpointer
np_double_p = ndpointer(dtype=np.float64)

In [12]:
dll = ctypes.cdll.LoadLibrary('./get_weights_c.so')
dll.get_weights_c.restype = None
dll.get_weights_c.argtypes = [
    np_double_p, np_double_p, np_double_p, ctypes.c_int, 
    np_double_p, np_double_p, np_double_p, np_double_p,
    np_double_p, np_double_p, np_double_p, np_double_p]

In [13]:
%%timeit -n 100

global weight_ijk, weight_i1jk, weight_ij1k, weight_i1j1k, \
weight_ijk1, weight_i1jk1, weight_ij1k1, weight_i1j1k1

(weight_ijk, weight_i1jk, weight_ij1k, weight_i1j1k,
weight_ijk1, weight_i1jk1, weight_ij1k1, weight_i1j1k1) = (
    np.empty_like(x) for _ in range(8)
)

dll.get_weights_c(
    x, y, z_beamframe, ctypes.c_int(len(x)),
    weight_ijk, weight_i1jk, weight_ij1k, weight_i1j1k,
    weight_ijk1, weight_i1jk1, weight_ij1k1, weight_i1j1k1
)

100 loops, best of 3: 59.8 ms per loop


# cython

In [14]:
%%cython --name get_weights_cy
# distutils: sources = ./get_weights_c.c
# distutils: extra_compile_args = -std=c99 -O3 -lm

import numpy as np
cimport numpy as np

cdef extern void get_weights_c(
        # inputs:
        double* x, double* y, double* z,
        int n,
        # outputs:
        double* weight_ijk, double* weight_i1jk,
        double* weight_ij1k, double* weight_i1j1k,
        double* weight_ijk1, double* weight_i1jk1,
        double* weight_ij1k1, double* weight_i1j1k1)

def get_weights(x, y, z):
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] x_c = np.ascontiguousarray(x, dtype=np.double)
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] y_c = np.ascontiguousarray(y, dtype=np.double)
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] z_c = np.ascontiguousarray(z, dtype=np.double)
    
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] weight_ijk = np.empty_like(x_c)
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] weight_i1jk = np.empty_like(x_c)
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] weight_ij1k = np.empty_like(x_c)
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] weight_i1j1k = np.empty_like(x_c)
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] weight_ijk1 = np.empty_like(x_c)
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] weight_i1jk1 = np.empty_like(x_c)
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] weight_ij1k1 = np.empty_like(x_c)
    cdef np.ndarray[np.double_t, ndim=1, mode="c"] weight_i1j1k1 = np.empty_like(x_c)
    
    get_weights_c(
        &x_c[0], &y_c[0], &z_c[0], len(x_c),
        &weight_ijk[0], &weight_i1jk[0],
        &weight_ij1k[0], &weight_i1j1k[0],
        &weight_ijk1[0], &weight_i1jk1[0],
        &weight_ij1k1[0], &weight_i1j1k1[0]
    )
    
    return (weight_ijk, weight_i1jk, weight_ij1k, weight_i1j1k,
            weight_ijk1, weight_i1jk1, weight_ij1k1, weight_i1j1k1)

In [15]:
import get_weights_cy

In [16]:
%%timeit -n 100

global weight_ijk, weight_i1jk, weight_ij1k, weight_i1j1k, \
weight_ijk1, weight_i1jk1, weight_ij1k1, weight_i1j1k1

(weight_ijk, weight_i1jk, weight_ij1k, weight_i1j1k,
weight_ijk1, weight_i1jk1, weight_ij1k1, weight_i1j1k1) = \
    get_weights_cy.get_weights(x, y, z_beamframe)

100 loops, best of 3: 61 ms per loop


# Conclusion

Both the `ctypes` and `cython` approach to access the shared `C` library yield comparable timings (the results fluctuate within $\approx 5$ ms).